In [581]:
# Dependencies
import numpy as np
import pandas as pd

In [582]:
# load CSV
schools="schools_complete.csv"
students="students_complete.csv"
students_pd=pd.read_csv(students)
schools_pd=pd.read_csv(schools)
schools_pd=schools_pd.rename(columns={'name':'school'})

In [583]:
# District Summary table
total_schools=len(schools_pd["school"])
total_students=len(students_pd["school"])
total_budget=schools_pd["budget"].sum()
average_math_score=round(students_pd["math_score"].mean(),6)
average_reading_score=round(students_pd["reading_score"].mean(),6)
passing_math=round(students_pd.loc[students_pd["math_score"]>=70]["math_score"].count()/total_students*100,6)
passing_reading=round(students_pd.loc[students_pd["reading_score"]>=70]["reading_score"].count()/total_students*100,6)
overall_passing_rate=round((passing_math+passing_reading)/2,6)

District_Summary=pd.DataFrame({"Total Schools":[total_schools],
                               "Total Students":[total_students],
                               "Total Budget":[total_budget],
                               "Average Math Score":[average_math_score],
                               "Avreage Reading Score":[average_reading_score],
                               "% Passing Math":[passing_math],
                               "% Passing Reading":[passing_reading],
                               "% Overall Passing Rate":[overall_passing_rate]
                              })

District_Summary=District_Summary[["Total Schools",
                               "Total Students",
                               "Total Budget",
                               "Average Math Score",
                               "Avreage Reading Score",
                               "% Passing Math",
                               "% Passing Reading",
                               "% Overall Passing Rate"]]

District_Summary["Total Budget"]=District_Summary["Total Budget"].map("${0:,.0f}".format)
District_Summary["Total Students"]=District_Summary["Total Students"].map("{0:,.0f}".format)
District_Summary.head()


,Total Schools,Total Students,Total Budget,Average Math Score,Avreage Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.985371,81.87784,74.980853,85.805463,80.393158


In [584]:
# Group student data by school
groupedStudent_School_df = students_pd.groupby(["school"], as_index=False)

# average Math score per school
avgMathScore_sch_df = pd.DataFrame(groupedStudent_School_df["math_score"].mean())

# average Reading score per school
avgReadingScore_sch_df = pd.DataFrame(groupedStudent_School_df["reading_score"].mean())
 
#Group math scores better than 70
group_mathPass = students_pd.loc[students_pd["math_score"]>70,:].groupby(['school'],as_index=False)
group_mathPass=pd.DataFrame(group_mathPass["math_score"].count())
group_mathPass.columns= ["school", "mathPassCnt"]

# total reading pass scores as a prep for the percent pass
group_readingPass = students_pd.loc[students_pd["reading_score"]>70,:].groupby(['school'],as_index=False)
group_readingPass=pd.DataFrame(group_readingPass["reading_score"].count())
group_readingPass.columns= ["school", "readingPassCnt"]

# Merge school table with grouped data
mergedStudentSchool_df = pd.merge(schools_pd, avgMathScore_sch_df, on="school" )
mergedStudentSchool_df = pd.merge(mergedStudentSchool_df, avgReadingScore_sch_df, on="school" )
mergedStudentSchool_df = pd.merge(mergedStudentSchool_df, group_mathPass, on="school" )
mergedStudentSchool_df = pd.merge(mergedStudentSchool_df, group_readingPass, on="school")

    
#calc per student budget
mergedStudentSchool_df["Per Student Budget"]=mergedStudentSchool_df["budget"]/mergedStudentSchool_df["size"]

# Add calculated columns for math and reading percent pass
mergedStudentSchool_df["% Passing Math"] = (mergedStudentSchool_df["mathPassCnt"]/mergedStudentSchool_df["size"])*100
mergedStudentSchool_df["% Passing Reading"] = (mergedStudentSchool_df["readingPassCnt"]/mergedStudentSchool_df["size"])*100
mergedStudentSchool_df["% Overall Passing Rate"]=(mergedStudentSchool_df["% Passing Math"]+mergedStudentSchool_df["% Passing Reading"])/2

# Save data unformatted for later use
school_summary_raw = pd.DataFrame(mergedStudentSchool_df[["school", "type", "size", "budget", "Per Student Budget", "math_score", "reading_score", "% Passing Math", "% Passing Reading" ,"% Overall Passing Rate"]])
school_summary_raw.columns = ["SchoolName", "School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]

# Select and rename columns
school_summary = pd.DataFrame(mergedStudentSchool_df[["school", "type", "size", "budget", "Per Student Budget", "math_score", "reading_score", "% Passing Math", "% Passing Reading" ,"% Overall Passing Rate"]])
school_summary.columns = ["SchoolName", "School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]

# Apply formatting to columns
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)

# Set the index of this new dataframe to be the SchoolName
school_summary = school_summary.set_index("SchoolName")
school_summary.head(len(school_summary))

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
SchoolName,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,89.892107,92.617831,91.254969
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,91.553134
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,71.965434
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183


In [585]:
# Create sorted list of top performing schools
sort_school_summary = school_summary.sort_values("% Overall Passing Rate", ascending=False)[:5]
sort_school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
SchoolName,,,,,,,,,
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,91.943867
Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,90.277778,93.444444,91.861111
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183


In [586]:
# Create sorted list of bottom performing schools
sort_school_summary = school_summary.sort_values("% Overall Passing Rate")[:5]
sort_school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
SchoolName,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,64.066017,77.744436,70.905226
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,71.067003
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098


In [587]:
# Get data by grades
ninth_df = students_pd.loc[students_pd["grade"] == "9th"].groupby("school", as_index=False)
tenth_df = students_pd.loc[students_pd["grade"] == "10th"].groupby("school", as_index=False)
eleventh_df = students_pd.loc[students_pd["grade"] == "11th"].groupby("school", as_index=False)
twelfth_df = students_pd.loc[students_pd["grade"] == "12th"].groupby("school", as_index=False)

#Group and get average math scores
ninth_AvgMath = pd.DataFrame(ninth_df["math_score"].mean())
tenth_AvgMath = pd.DataFrame(tenth_df["math_score"].mean())
eleventh_AvgMath = pd.DataFrame(eleventh_df["math_score"].mean())
twelfth_AvgMath = pd.DataFrame(twelfth_df["math_score"].mean())

#Merge to Math score summary table
mathByGrade = pd.merge(ninth_AvgMath, tenth_AvgMath, on="school")
mathByGrade = pd.merge(mathByGrade, eleventh_AvgMath, on="school")
mathByGrade = pd.merge(mathByGrade, twelfth_AvgMath, on="school")
mathByGrade.columns = ["school","9th","10th","11th","12th"]
mathByGrade = mathByGrade.set_index("school")
mathByGrade.head(len(mathByGrade))

,9th,10th,11th,12th
school,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [588]:
#Group and get average reading scores
ninth_AvgRead = pd.DataFrame(ninth_df["reading_score"].mean())
tenth_AvgRead = pd.DataFrame(tenth_df["reading_score"].mean())
eleventh_AvgRead = pd.DataFrame(eleventh_df["reading_score"].mean())
twelfth_AvgRead = pd.DataFrame(twelfth_df["reading_score"].mean())

#Merge to Reading score summary table
readByGrade = pd.merge(ninth_AvgRead, tenth_AvgRead, on="school")
readByGrade = pd.merge(readByGrade, eleventh_AvgRead, on="school")
readByGrade = pd.merge(readByGrade, twelfth_AvgRead, on="school")
readByGrade.columns = ["school","9th","10th","11th","12th"]
readByGrade = readByGrade.set_index("school")
readByGrade.head(len(readByGrade))

,9th,10th,11th,12th
school,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [589]:
# Scores by School Spending
bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585 to 615", "$615 to 645", "$645 to 675"]
scoreByBudget = school_summary_raw[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]].groupby(pd.cut(school_summary_raw["Per Student Budget"], bins=bins, labels=group_names )).mean()
scoreByBudget.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
Per Student Budget,,,,,
<$585,83.455399,83.933814,90.350436,93.325838,91.838137
$585 to 615,83.599686,83.885211,90.788049,92.410786,91.599418
$615 to 645,79.079225,81.891436,73.021426,83.214343,78.117884
$645 to 675,76.997210,81.027843,63.972368,78.427809,71.200088


In [590]:
# Scores by School Size
bins = [0, 1000, 2500, 50000]
group_names = ["Small <(1000)", "Medium (1000 to 2500)", "Large (2500 to 5000)"]
scoreBySize = school_summary_raw[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]].groupby(pd.cut(school_summary_raw["Total Students"], bins=bins, labels=group_names)).mean()
scoreBySize.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
Total Students,,,,,
Small <(1000),83.821598,83.929843,91.158155,92.471895,91.815025
Medium (1000 to 2500),83.357937,83.885280,90.098249,93.246451,91.672350
Large (2500 to 5000),76.956733,80.966636,64.302528,78.324559,71.313543


In [591]:
# Scores by School Type
school_summary_raw["School Type"] = school_summary_raw["School Type"].replace({"Charter": 1, "District":2})
bins = [0, 1, 2]
group_names = ["Charter", "District"]
scoreByType = school_summary_raw[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]].groupby(pd.cut(school_summary_raw["School Type"], bins=bins,labels=group_names)).mean()
scoreByType.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
School Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,91.708019
District,76.956733,80.966636,64.302528,78.324559,71.313543
